## Load Packages

In [1]:
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# From the Chapter on Mean-Variance Analysis

In [3]:
function MVTangencyP(μ,Σ,Rf)           #calculates the tangency portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     #expected excess returns            
    Σ_1   = inv(Σ)
    w     = Σ_1 *μe/(oneV'Σ_1*μe)[1]
    muT   = (w'μ + (1-sum(w))*Rf)[1]
    StdRT = sqrt((w'Σ*w)[1])
    return w,muT,StdRT
end

MVTangencyP (generic function with 1 method)

# Three Assets and the Tangency Portfolio

In [4]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;          #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("expected returns: ")
printmat(μ)
println("covariance matrix:")
printmat(Σ)

(wT,muT,StdRT) = MVTangencyP(μ,Σ,Rf)
println("\nTangency portfolio: ")
printmat(wT)
println("mean and std of tangency portfolio: ")
printmat([muT StdRT])

expected returns: 
     0.115
     0.095
     0.060

covariance matrix:
     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010


Tangency portfolio: 
     0.248
     0.682
     0.070

mean and std of tangency portfolio: 
     0.097     0.074



# β of Three Portfolios

Recall: $\beta_i = \textrm{Cov}(R_i,R_m)/\textrm{Var}(R_m)$.

Let $\Sigma$ be the covariance matrix and $\mu$ the vector of expected returns of the investible assets $R$. 

Asset $i$ is the portfolio of those assets $R_i = w_i'R$.  

Notice that $ER_i = w_i'\mu$ and $\textrm{Cov}(R_i,R_j) = w_i'\Sigma w_j$

In [5]:
w = [ 1   0  0;          #3 different (trivial) portfolios
      0   1  0;
      0   0   1]   
printmat(w)

n = size(w,2)
ERi   = fill(NaN,n)           #expected returns
CovRm = fill(NaN,n)           #covariance with tangency portfolio
β     = fill(NaN,n)  
for i = 1:n
    ERi[i]   = (w[:,i]'μ)[1]
    CovRm[i] = (w[:,i]'Σ*wT)[1]
    β[i]     = CovRm[i]/StdRT^2  #usual OLS coefficient
end    
println("\nERi,Cov(Ri,Rm), β for $n portfolios: ")
printmat([ERi CovRm β])

         1         0         0
         0         1         0
         0         0         1


ERi,Cov(Ri,Rm), β for 3 portfolios: 
     0.115     0.007     1.259
     0.095     0.005     0.963
     0.060     0.002     0.444



# Trying CAPM on the Three Portfolios

Recall: CAPMS says $\textrm{E}R_{i}^{e}  =\beta_{i}\mu_{T}^{e}$, or

 $\textrm{E}R_{i}  = R_f + \beta_{i}(\mu_{T}-R_f)$

In [6]:
ERi_CAPM = fill(NaN,n)           #expected returns according to CAPM
for i = 1:n
    ERi_CAPM[i] = Rf + (β[i]*(muT-Rf))[1]
end    
println("\nERi, and ERi as suggested by CAPM: ")
printmat([ERi ERi_CAPM])


ERi, and ERi as suggested by CAPM: 
     0.115     0.115
     0.095     0.095
     0.060     0.060



In [7]:
scatter(β,ERi_CAPM*100,xlims=(0,2),ylims=(0,15),legend=false)
title!("beta vs ERi")
xlabel!("beta")
ylabel!("ERi, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 beta vs ERi 
 
 
 beta 
 
 
 ERi, %

# Empirical Test of CAPM: Loading Data

In [8]:
xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
x    = xx[1]
Rme  = x[:,2]                #market excess return
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate


x = readdlm("Data/FF25Ps.csv",',')  #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1;7;13;19;25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                 #no. obs and  no. test assets

(388, 5)

# OLS Estimation and Testing α = 0

Recall: estimate $(\alpha_{i},b_{i})$ in the CAPM regression

$R_{it}^{e}  =\alpha_{i}+b_{i}R_{mt}^{e}+\varepsilon_{it}$

Test if $\alpha_{i}=0$

Under the Gauss-Markov assumptions, $\textrm{Cov}(\alpha_{i},b_{i}) = (X'X)^{-1}\textrm{Var}(\varepsilon_{it})$,
where row $t$ of $X$ contains $[1, R_{mt}^e]$. Use element [1,1] of the covariance matrix as the variance of the $\alpha_i$ estimate

In [9]:
c   = ones(T,1)                 #a vector with ones
x   = [c Rme]                   #regressors
Σ_1 = inv(x'x)

α     = fill(NaN,n)
tstat = fill(NaN,n)
for i = 1:n                              #loop over the different test assets
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    α[i]      = b_i[1]
    Covb      = Σ_1*var(residual)   
    tstat[i] = (α[i]-0)/sqrt(Covb[1,1])
end    
println("\nOLS intercepts and t-stats, regressing Re on constant and Rme")
printmat([α tstat])


OLS intercepts and t-stats, regressing Re on constant and Rme
    -0.504    -1.656
     0.153     1.031
     0.305     2.471
     0.279     2.163
     0.336     2.073

